In [ ]:
import gurobipy as gp;
from gurobipy import GRB;

In [ ]:
# Input data
Yield=[[2.5, 3, 20],
       [3, 3.6, 24],
       [2, 2.4, 16]];
Cost=[150, 230, 260];
Price = [170, 150, 36];
OverPrice = 10; # Only for beans
Oversize = 6000; # Only for beans
ExtraPrice = [238,210]; # Only for the first two crops
Demand = [200,240];
Land = 500;
numScens = len(Yield);
numCrops = len(Cost);

meanYield = [0.0]*numScens; # Careful - this technique doesn't generalize to multidimensional arrays or lists of lists

for j in range(numCrops):
    for k in range(numScens):
        meanYield[j] = meanYield[j]+Yield[k][j];
    meanYield[j] = meanYield[j]*1.0/numScens;


In [ ]:
m = gp.Model("farmer");
# Set the sense for the objective: we'd like to maximize
m.modelSense = GRB.MAXIMIZE; # this is to maximize the total profit

In [ ]:
# acres of wheat, corn, beans planted
x = {};
for i in range(numCrops):
   x[i] = m.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=-Cost[i]);

# tons of wheat, corn, beans sold at regular price
w = {};
for i in range(numCrops):
   w[i] = m.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=Price[i]);

# tons of beans sold at extra price
e = m.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=OverPrice);

# tons of wheat corn purchased from super farmer
y = {};
for i in range(numCrops-1):
    y[i] = m.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=-ExtraPrice[i]);


In [ ]:
# Add constraints
m.addConstr(sum(x[i] for i in range(numCrops)) <= Land);
m.addConstr(meanYield[0]*x[0]+y[0]-w[0] == Demand[0]);
m.addConstr(meanYield[1]*x[1]+y[1]-w[1] == Demand[1]);
m.addConstr(meanYield[2]*x[2]-w[2]-e == 0);
m.addConstr(w[2]<=Oversize);
# Finish adding all variables and constraints, do an update to make sure things are up to date.
m.update();
m.setParam("OutputFlag", 0); # disable output information unless you want to take a look at the solution process

In [ ]:
m.optimize();
if m.status == GRB.OPTIMAL:
    print('\nProfit: %g' % m.objVal);
    print('\nPlant:');
    plant_x_MVP = m.getAttr('x', x);
    for i in range(numCrops):
        if x[i].x > 0.0001:
            print('%s: %g' % (i, plant_x_MVP[i]));
else:
    print('No solution');

We now solve the two-stage SP model, create multiple copies of second-stage variables, one copy per scenario.

In [ ]:
m_2SP = gp.Model("farmer-two-stage-SP");
m_2SP.modelSense = GRB.MAXIMIZE; # this is to maximize the total expected profit
# acres of wheat, corn, beans planted
x = {};
for i in range(numCrops):
   x[i] = m_2SP.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=-Cost[i]);

# tons of wheat, corn, beans sold at regular price
ww = {};
for k in range(numScens):
    ww[k] = {};
    for i in range(numCrops):
        ww[k][i] = m_2SP.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=1/numScens*Price[i]);

# tons of beans sold at extra price
ee = {};
for k in range(numScens):
    ee[k] = m_2SP.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=1/numScens*OverPrice);

# tons of wheat corn purchased from super farmer
yy = {};
for k in range(numScens):
    yy[k] = {};
    for i in range(numCrops-1):
        yy[k][i] = m_2SP.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=-1/numScens*ExtraPrice[i]);


# First-stage constraint
m_2SP.addConstr(sum(x[i] for i in range(numCrops)) <= Land);
# Second-stage constraint
for k in range(numScens):
    m_2SP.addConstr(Yield[k][0]*x[0]+yy[k][0]-ww[k][0] == Demand[0]);
    m_2SP.addConstr(Yield[k][1]*x[1]+yy[k][1]-ww[k][1] == Demand[1]);
    m_2SP.addConstr(Yield[k][2]*x[2]-ww[k][2]-ee[k] == 0);
    m_2SP.addConstr(ww[k][2]<=Oversize);
    
m_2SP.update();
m_2SP.setParam("OutputFlag", 0); # disable output information unless you want to take a look at the solution process

In [ ]:
m_2SP.optimize();
if m_2SP.status == GRB.OPTIMAL:
    print('\nExpected Profit: %g' % m_2SP.objVal);
    print('\nPlant:');
    plant_x_SP = m_2SP.getAttr('x', x);
    for i in range(numCrops):
        if x[i].x > 0.0001:
            print('%s: %g' % (i, plant_x_SP[i]));
else:
    print('No solution');
    
SPObjVal = m_2SP.objVal;

We next calculate EVPI. To do so, solve all the scenario-based problems separately, and then take the average. Then compare the average with SP objective value.

In [ ]:
EVPIScenObjVal = [0.0]*numScens;
m_scen = {};
x = {};
w = {};
e = {};
y = {};

for k in range(numScens):
    m_scen[k] = gp.Model("farmer-2nd-prob"+str(k));
    m_scen[k].modelSense = GRB.MAXIMIZE; # this is to maximize the total expected profit
    # acres of wheat, corn, beans planted
    x[k] = {};
    for i in range(numCrops):
       x[k][i] = m_scen[k].addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=-Cost[i]);

    # tons of wheat, corn, beans sold at regular price
    w[k] = {};
    for i in range(numCrops):
        w[k][i] = m_scen[k].addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=Price[i]);

    # tons of beans sold at extra price
    e[k] = m_scen[k].addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=OverPrice);

    # tons of wheat corn purchased from super farmer
    y[k] = {};
    for i in range(numCrops-1):
        y[k][i] = m_scen[k].addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=-ExtraPrice[i]);

    m_scen[k].addConstr(sum(x[k][i] for i in range(numCrops)) <= Land);
    m_scen[k].addConstr(Yield[k][0]*x[k][0]+y[k][0]-w[k][0] == Demand[0]);
    m_scen[k].addConstr(Yield[k][1]*x[k][1]+y[k][1]-w[k][1] == Demand[1]);
    m_scen[k].addConstr(Yield[k][2]*x[k][2]-w[k][2]-e[k] == 0);
    m_scen[k].addConstr(w[k][2]<=Oversize);
    
    m_scen[k].update();
    m_scen[k].setParam("OutputFlag", 0); # disable output information unless you want to take a look at the solution process
    m_scen[k].optimize();
    if m_scen[k].status == GRB.OPTIMAL:
        EVPIScenObjVal[k] = m_scen[k].objVal;
    else:
        print('No solution');
        exit(0);

PIobjVal = sum(EVPIScenObjVal)*1.0/numScens;
print("EVPI = ",PIobjVal-SPObjVal);

We next compute VSS. To do so, we get the first-stage solution from the MVP, and evaluate it using the objective function of the SP.

In [ ]:
# remember that the first-stage solution from the MVP is recorded as plant_x_MVP (defined above)
MVPScenObjVal = [0.0]*numScens;
for k in range(numScens):
    for i in range(numCrops):
        m_scen[k].addConstr(x[k][i] == plant_x_MVP[i]);
    m_scen[k].update();
    m_scen[k].optimize();
    if m_scen[k].status == GRB.OPTIMAL:
        MVPScenObjVal[k] = m_scen[k].objVal;
    else:
        print('No solution');
        exit(0);
        
MVPObjVal = sum(MVPScenObjVal)*1.0/numScens;
print("VSS = ", SPObjVal-MVPObjVal);